<a href="https://colab.research.google.com/github/Strojove-uceni/23206-final-sign-language-recognition-in-video/blob/main/NN_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import os
import math
import sys
import cv2
import io

from google.colab import drive

In [2]:
drive.mount('/content/drive')

path = "/content/drive/MyDrive/Colab Notebooks/SU2/data"

Mounted at /content/drive


In [3]:
!git clone https://github.com/jiri99/SU_sign-language.git

Cloning into 'SU_sign-language'...
remote: Enumerating objects: 8, done.
remote: Counting objects: 100% (8/8), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 8 (delta 1), reused 4 (delta 0), pack-reused 0
Receiving objects: 100% (8/8), 5.94 KiB | 5.94 MiB/s, done.
Resolving deltas: 100% (1/1), done.


In [7]:
repo_path = "/content/SU_sign_language/"
sys.path.append(os.path.abspath(repo_path))

In [8]:
from buffer import ParquetData

FileNotFoundError: ignored

In [ ]:
selected_landmark_indices = [33, 133, 159, 263, 46, 70, 4, 454, 234, 10, 338, 297, 332, 61, 291, 0, 78, 14, 317,
                             152, 155, 337, 299, 333, 69, 104, 68, 398]

df_train = pd.read_csv(path + "/train_mod.csv", sep=",")
df_train.head()

data_load = ParquetData()
data_load.read_all(path, df_train, selected_landmark_indices, 37)